In [110]:
import numpy as np
from scipy.optimize import linprog

In [111]:
# set seed for reproducibility
np.random.seed(23)


In [112]:
import numpy as np

def create_transportation_problem(num_sources, num_destinations, zero_chance=0.1, M_chance=0.05, M_value=999):
    """
    Creates a transportation problem with specified number of sources and destinations.
    Costs can be randomly set to zero or a very high number (M_value) with given chances.
    
    Parameters:
    num_sources (int): Number of sources.
    num_destinations (int): Number of destinations.
    zero_chance (float): Probability of a cost being zero.
    M_chance (float): Probability of a cost being M_value (very high cost).
    M_value (int): The very high cost to use for prohibitive connections.
    
    Returns:
    tuple: Returns a tuple containing the cost matrix, supply, and demand arrays.
    """
    
    if num_sources <= 0 or num_destinations <= 0:
        raise ValueError("Number of sources and destinations must be positive integers")
    
    # Random cost matrix with occasional zeros
    cost_matrix = np.random.randint(1, 11, size=(num_sources, num_destinations))
    zero_mask = np.random.rand(num_sources, num_destinations) < zero_chance
    M_mask = np.random.rand(num_sources, num_destinations) < M_chance
    cost_matrix[zero_mask] = 0
    cost_matrix[M_mask] = M_value

    # Random supply and demand ensuring total supply equals total demand
    total_units = np.random.randint(20, 50) * num_sources
    supply = np.random.randint(1, total_units // num_sources, size=num_sources)
    demand = np.random.randint(1, total_units // num_destinations, size=num_destinations)
    
    # Adjust supply and demand to introduce occasional zero rows/columns
    if np.random.rand() < zero_chance:
        supply[np.random.randint(0, num_sources)] = 0
    if np.random.rand() < zero_chance:
        demand[np.random.randint(0, num_destinations)] = 0
    
    # Ensure total supply equals total demand after zero introduction
    supply_demand_difference = supply.sum() - demand.sum()
    if supply_demand_difference > 0:
        demand[-1] += supply_demand_difference
    else:
        supply[-1] -= supply_demand_difference

    return cost_matrix, supply, demand

In [113]:
def north_west_corner_rule(supply, demand):
    """
    Determines the basic variable indices using the North-West Corner rule for the
    transportation problem.
    
    Parameters:
    supply (np.array): The supply array.
    demand (np.array): The demand array.
    
    Returns:
    list of tuples: Returns the indices of the basic variables.
    """
    supply = supply.copy()
    demand = demand.copy()
    num_sources = len(supply)
    num_destinations = len(demand)
    
    basic_variables_indices = []
    i, j = 0, 0
    
    while i < num_sources and j < num_destinations:
        # Assign as much as possible to the current cell
        amount = min(supply[i], demand[j])
        basic_variables_indices.append((i, j))
        
        # Decrease the supply and demand
        supply[i] -= amount
        demand[j] -= amount
        
        # Move to next row or column
        if demand[j] == 0 and j < num_destinations:
            j += 1
        elif supply[i] == 0 and i < num_sources:
            i += 1
            
    return basic_variables_indices


In [114]:
def create_boolean_matrix(num_sources, num_destinations, basic_variables_indices):
    # Initialize a boolean matrix with False values
    bool_matrix = np.zeros((num_sources, num_destinations), dtype=bool)
    # Set the positions of the basic variables to True
    for i, j in basic_variables_indices:
        bool_matrix[i, j] = True
    return bool_matrix

In [115]:
def calculate_uv(basic_variables_indices, cost_matrix):
    # Initialize u and v arrays with None to indicate uncalculated variables
    num_sources, num_destinations = cost_matrix.shape
    u = [None] * num_sources
    v = [None] * num_destinations
    
    # Set the first u to 0 as a starting point for calculations
    u[0] = 0
    
    # Continue until all u and v are calculated
    while None in u or None in v:
        for i, j in basic_variables_indices:
            if u[i] is not None and v[j] is None:
                v[j] = cost_matrix[i, j] - u[i]
            elif u[i] is None and v[j] is not None:
                u[i] = cost_matrix[i, j] - v[j]
    
    return u, v


In [116]:
def find_entering_variable(cost_matrix, u, v, basic_variables_indices):
    # Determine the reduced costs for all non-basic variables
    num_sources, num_destinations = cost_matrix.shape
    reduced_costs = np.zeros_like(cost_matrix)
    
    for i in range(num_sources):
        for j in range(num_destinations):
            if (i, j) not in basic_variables_indices:
                reduced_costs[i, j] = cost_matrix[i, j] - u[i] - v[j]
    
    # Select the non-basic variable with the highest reduced cost for entry
    entering_indices = np.argwhere(reduced_costs > 0)
    if entering_indices.size == 0:
        return None  # Optimal solution found
    
    entering_index = entering_indices[reduced_costs[entering_indices[:, 0], entering_indices[:, 1]].argmax()]
    return tuple(entering_index)


In [117]:
def find_loop(basic_variables_indices, entering_variable):
    num_sources, num_destinations = basic_variables_indices.shape
    # Initialize loop matrix which contains the same shape as basic variables but with boolean values
    loop = np.zeros_like(basic_variables_indices, dtype=bool)

    def is_connected_to_entering(i, j):
        return i == entering_variable[0] or j == entering_variable[1]

    def find_path(i, j, direction):
        loop[i, j] = True
        if is_connected_to_entering(i, j) and (i, j) != entering_variable:
            return True
        if direction != 'up' and i + 1 < num_sources and basic_variables_indices[i + 1, j] and not loop[i + 1, j]:
            if find_path(i + 1, j, 'down'):
                return True
        if direction != 'left' and j + 1 < num_destinations and basic_variables_indices[i, j + 1] and not loop[i, j + 1]:
            if find_path(i, j + 1, 'right'):
                return True
        if direction != 'down' and i - 1 >= 0 and basic_variables_indices[i - 1, j] and not loop[i - 1, j]:
            if find_path(i - 1, j, 'up'):
                return True
        if direction != 'right' and j - 1 >= 0 and basic_variables_indices[i, j - 1] and not loop[i, j - 1]:
            if find_path(i, j - 1, 'left'):
                return True
        loop[i, j] = False
        return False

    if not find_path(*entering_variable, ''):
        return None  # No loop found

    loop_indices = np.argwhere(loop)
    # Construct a path that alternates between basic and non-basic variables.
    return loop_indices

In [118]:
def update_bfs(bfs, loop, basic_variables_indices):
    # Identify the leaving variable as the one with the smallest value in the loop
    leaving_variable_value = float('inf')
    leaving_variable_index = None
    for index in loop:
        if index in basic_variables_indices:
            i, j = index
            if bfs[i, j] < leaving_variable_value:
                leaving_variable_value = bfs[i, j]
                leaving_variable_index = index

    # Update the BFS by adding/subtracting the leaving variable value along the loop
    sign = 1  # The sign will alternate starting with +1
    for index in loop:
        i, j = index
        bfs[i, j] += sign * leaving_variable_value
        sign *= -1  # Alternate the sign

    # Remove the leaving variable and add the entering variable to the set of basic variables
    new_basic_variables_indices = set(basic_variables_indices)
    new_basic_variables_indices.remove(leaving_variable_index)
    new_basic_variables_indices.add(loop[0])  # Assuming loop[0] is the entering variable's index

    return bfs, new_basic_variables_indices, leaving_variable_index


In [119]:
def calculate_total_cost(bfs, cost_matrix):
    total_cost = np.sum(bfs * cost_matrix)  # Element-wise multiplication and sum
    return total_cost

In [133]:
num_sources, num_destinations = 5, 5
cost_matrix, supply, demand = create_transportation_problem(num_sources, num_destinations)
supply_copy = supply.copy()
demand_copy = demand.copy()

# Initialize BFS using the North West Corner Rule
bfs = np.zeros((num_sources, num_destinations))
basic_variables_indices = north_west_corner_rule(supply, demand)

# Fill the initial BFS using the basic variables
for i, j in basic_variables_indices:
    allocation = min(supply[i], demand[j])
    bfs[i][j] = allocation
    supply[i] -= allocation
    demand[j] -= allocation

iteration = 0  # Iteration counter

# Calculate the initial total cost
z = calculate_total_cost(bfs, cost_matrix)
print(f"Iteration: {iteration}, Z: {z}")

while True and not iteration > 200:
    iteration += 1
    # Step 1: Calculate Dual Variables (u and v)
    u, v = calculate_uv(basic_variables_indices, cost_matrix)
    
    # Step 2: Find Entering Variable
    entering_variable = find_entering_variable(cost_matrix, u, v, basic_variables_indices)
    
    # If no entering variable is found, the current BFS is optimal
    if not entering_variable:
        print("Optimal solution found.")
        break
    
    # Convert basic_variables_indices to boolean matrix format for find_loop function
    bool_basic_variables = create_boolean_matrix(num_sources, num_destinations, basic_variables_indices)

    # Step 3: Find Loop for the entering variable
    loop = find_loop(bool_basic_variables, entering_variable)
    
    if loop is None:
        print("No loop found - problem may be degenerate or there is an error.")
        break
    
    # Step 4: Update the BFS using the identified loop
    bfs, basic_variables_indices, leaving_variable = update_bfs(bfs, loop, basic_variables_indices)
    
    # Calculate the total cost for the current BFS
    z = calculate_total_cost(bfs, cost_matrix)
    print(f"Iteration: {iteration}, Z: {z}")

# At this point, bfs contains the optimal allocations, u and v contain the final dual variables.
print("Final BFS:\n", bfs)
print("Final Z value:", z)
print("Final Dual Variables (u, v):", u, v)

Iteration: 0, Z: 439.0
No loop found - problem may be degenerate or there is an error.
Final BFS:
 [[18.  0.  0.  0.  0.]
 [ 3. 18.  0.  0.  0.]
 [ 0.  4. 11.  0.  0.]
 [ 0.  0.  8.  7.  8.]
 [ 0.  0.  0.  0. 15.]]
Final Z value: 439.0
Final Dual Variables (u, v): [0, -5, 2, 5, 4] [9, 5, 5, 2, -3]


In [134]:
import pulp

def solve_with_pulp(cost_matrix, supply, demand):
    num_sources, num_destinations = cost_matrix.shape
    problem = pulp.LpProblem("Transportation_Problem", pulp.LpMinimize)

    # Create variables for flow on transportation arcs
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_sources) for j in range(num_destinations)),
                              lowBound=0)

    # Objective function: Minimize total transportation cost
    problem += pulp.lpSum([cost_matrix[i][j] * x[(i, j)] for i in range(num_sources) for j in range(num_destinations)])

    # Constraints for supply
    for i in range(num_sources):
        problem += pulp.lpSum([x[(i, j)] for j in range(num_destinations)]) == supply[i]

    # Constraints for demand
    for j in range(num_destinations):
        problem += pulp.lpSum([x[(i, j)] for i in range(num_sources)]) == demand[j]

    # Solve the problem
    problem.solve()

    # Return the objective function value and the solution (if it's found)
    if problem.status == pulp.LpStatusOptimal:
        solution = {(i, j): x[(i, j)].varValue for i in range(num_sources) for j in range(num_destinations)}
        return pulp.value(problem.objective), solution
    else:
        return None, None

# Solve using PuLP and compare with your solution
pulp_cost, pulp_solution = solve_with_pulp(cost_matrix, supply_copy, demand_copy)

# Check if the total costs are the same
print("PuLP solution total cost:", pulp_cost)

# You can also print the solution matrices for a visual comparison, if necessary.


PuLP solution total cost: 165.0
